In [1]:
import pandas as pd
import numpy as np
import re
df=pd.read_csv('../Emission_computing/final_ingredients_emissions.csv',error_bad_lines=False)
from os import name
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
import re
nltk.download('wordnet')

/home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 280: expected 2 fields, saw 4\n'
[nltk_data] Downloading package wordnet to /home/mateo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df

,ingredient,emissions
0,allspice,1133.0
1,almond extract,656.0
2,almonds,2856.0
3,anchovy,4220.0
4,apple juice,2440.0
...,...,...
555,bok choy,622.0
556,figs,192.0
557,mung bean sprouts,428.0
558,soup mix,428.0


In [3]:
output_dict={'ingredient': ['ground beef', 'fresh ginger', 'garlic', 'cilantro', 'limes', 'salt', 'chile paste', 'red onion', 'sea bass', 'chili', 'cilantro', 'chili', 'sweet potato', 'fine sea salt'], 'value': [1, 0.06, 0.25, 1.0, 2.0, 0.12, 0.5, 0.25, 0.33, 0.0, 0.25, 0.25, 0.25, 0.0], 'metric': ['pound', 'piece', 'clove', 'sprig', '<unit>', 'teaspoon', 'teaspoon', '<unit>', 'pound', None, 'sprig', '<unit>', '<unit>', None]}

In [4]:
output_dict

{'ingredient': ['ground beef',
  'fresh ginger',
  'garlic',
  'cilantro',
  'limes',
  'salt',
  'chile paste',
  'red onion',
  'sea bass',
  'chili',
  'cilantro',
  'chili',
  'sweet potato',
  'fine sea salt'],
 'value': [1,
  0.06,
  0.25,
  1.0,
  2.0,
  0.12,
  0.5,
  0.25,
  0.33,
  0.0,
  0.25,
  0.25,
  0.25,
  0.0],
 'metric': ['pound',
  'piece',
  'clove',
  'sprig',
  '<unit>',
  'teaspoon',
  'teaspoon',
  '<unit>',
  'pound',
  None,
  'sprig',
  '<unit>',
  '<unit>',
  None]}

In [5]:
missing_values = ['None']

In [6]:
def fill_empties(output_dict):
    output_df=pd.DataFrame(output_dict)
    output_df.metric=output_df.metric.astype(str)
    for item in missing_values:
        output_df.metric.replace(item, "g", inplace=True)
    return output_df

In [7]:
output_df=fill_empties(output_dict)

In [69]:
output_df

,ingredient,value,metric
0,ground beef,1.00,pound
1,fresh ginger,0.06,piece
2,garlic,0.25,clove
3,cilantro,1.00,sprig
4,limes,2.00,<unit>
5,salt,0.12,teaspoon
6,chile paste,0.50,teaspoon
7,red onion,0.25,<unit>
8,sea bass,0.33,pound
9,chili,0.00,g


In [108]:
def match_ingredients(output_df):
    final_dict={'ingredient':[], 'emission':[],'value':[],'metric':[]}
    missing_ingredients={'missing_ingredients':[]}
    
    for ingredient in output_df['ingredient']:
        print(ingredient)
        
        # 1. Lemmatize 
        ingredient_words=re.split('\s+', ingredient)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
        ingredient_words=re.split('\s+', lemmatized_output)
        # "ground beef"
        
        # 2. Try the whole lemmatized sentence
        if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
                final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
                final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
                final_dict['value'].append(output_df[output_df["ingredient"]==ingredient].value.iloc[0])
                final_dict['metric'].append(output_df[output_df["ingredient"]==ingredient].metric.iloc[0])
        # False
        
        # 3. If the lemmatized output does not work, try the words
        else:
            for word in ingredient_words:
                # "ground" 
                # "beef"
                try:
                    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].ingredient.values[0])
                    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].emissions.values[0])
                    final_dict['value'].append(output_df[output_df["ingredient"]==ingredient].value.iloc[0])
                    final_dict['metric'].append(output_df[output_df["ingredient"]==ingredient].metric.iloc[0])                    
                except IndexError:
                    missing_ingredients['missing_ingredients'].append(ingredient)
            
#     final_df=pd.DataFrame.from_dict(final_dict)
#     # Adding the columns "value" and metric from output_dict
#     final_df=final_df.merge(output_df,on="ingredient")
    final_df=pd.DataFrame(final_dict)

    return final_df,missing_ingredients

In [91]:
ingredient="ground beef"

In [92]:
final_dict={'ingredient':[], 'emission':[],'value':[],'metric':[]}

In [93]:
final_dict['value'].append(output_df[output_df["ingredient"]==ingredient]['value'][0])

In [94]:
final_dict

{'ingredient': [], 'emission': [], 'value': [1.0], 'metric': []}

In [96]:
output_df

,ingredient,value,metric
0,ground beef,1.00,pound
1,fresh ginger,0.06,piece
2,garlic,0.25,clove
3,cilantro,1.00,sprig
4,limes,2.00,<unit>
5,salt,0.12,teaspoon
6,chile paste,0.50,teaspoon
7,red onion,0.25,<unit>
8,sea bass,0.33,pound
9,chili,0.00,g


In [107]:
output_df[output_df['ingredient']=='fresh ginger']['value'].iloc[0]

0.06

In [109]:
final_df,missing_ingredients=match_ingredients(output_df)

ground beef
fresh ginger
garlic
cilantro
limes
salt
chile paste
red onion
sea bass
chili
cilantro
chili
sweet potato
fine sea salt


In [110]:
final_df

,ingredient,emission,value,metric
0,groundnut oil,900.0,1.00,pound
1,beef,43330.0,1.00,pound
2,fresh ginger,452.0,0.06,piece
3,garlic,452.0,0.25,clove
4,cilantro,1133.0,1.00,sprig
5,lime,1900.0,2.00,<unit>
6,salt,1133.0,0.12,teaspoon
7,chile paste,1535.0,0.50,teaspoon
8,red onion,1051.0,0.25,<unit>
9,sea bass,4362.0,0.33,pound


In [111]:
def convert(final_df):
    output_df=final_df.merge(conv_df, on='metric')
    output_df.emission=output_df.emission.round()
    output_df["calculated gCO2e"]=output_df["emission"]*output_df["value"]*output_df["to_Kg_multiplier"]
    return output_df
print('OK step 3')

OK step 3


In [118]:
conversion_file_path='../Emission_computing/conversion.csv'
conv_df=pd.read_csv(conversion_file_path)

In [119]:
conv_df

,metric,to_Kg_multiplier
0,gram,0.001000
1,milliliter,0.001000
2,teaspoon,0.005000
3,tablespoon,0.015000
4,pound,0.454545
5,ounce,0.028349
6,pinch,0.003000
7,squid,0.063000
8,lemon,0.070000
9,loaf,0.400000


In [120]:
convert(final_df)

,ingredient,emission,value,metric,to_Kg_multiplier,calculated gCO2e
0,groundnut oil,900.0,1.00,pound,0.454545,409.090909
1,beef,43330.0,1.00,pound,0.454545,19695.454545
2,sea bass,4362.0,0.33,pound,0.454545,654.300000
3,garlic,452.0,0.25,clove,0.006000,0.678000
4,cilantro,1133.0,1.00,sprig,0.005000,5.665000
5,cilantro,1133.0,1.00,sprig,0.005000,5.665000
6,lime,1900.0,2.00,<unit>,1.000000,3800.000000
7,red onion,1051.0,0.25,<unit>,1.000000,262.750000
8,sweet potato,1273.0,0.25,<unit>,1.000000,318.250000
9,salt,1133.0,0.12,teaspoon,0.005000,0.679800


In [123]:
final_result=convert(final_df)["calculated gCO2e"].sum()

In [131]:
final_result=round(final_result)

In [132]:
final_result

25156

In [126]:
miles_per_Kg = round(final_result*0.001*(296/116),2)
heating_per_Kg = round(final_result*0.001*(29/116),2)
showers_per_Kg = round((final_result*0.001*(18/116)),2)
stream_hrs_kg= round(final_result*0.001*(1/float(55/1000)),2)

In [127]:
miles_per_Kg

64.19

In [128]:
heating_per_Kg

6.29

In [129]:
showers_per_Kg

3.9

In [130]:
stream_hrs_kg

457.39